## Membaca file

In [3]:
import pandas as pd

# Membaca file 'hasil_labelling_emas.csv'
df_label = pd.read_csv('hasil_labelling_emas.csv')

# Membaca file 'scraping_emas.csv'
df_scraping = pd.read_csv('scraping_emas.csv')

# Menampilkan 5 baris pertama untuk memastikan data terbaca
print("Data Labelling:")
print(df_label.head())

print("\nData Scraping:")
print(df_scraping.head())

Data Labelling:
                                                text    label
0  Tungguin aja bakal balik ke bawah 2 juta per g...  negatif
1               Emas ada bentuk fisik nya\nBitcoin ?   netral
2  Lu makan aja tuh emas kalau bisa kenyang lu !!...  negatif
3  Bukan palsu tetpi emas kurang .metal LG LG tin...  negatif
4                      LM Antam habis di pegadaian 🤭  negatif

Data Scraping:
            username                waktu  \
0       @tribudi2562  2025-10-23 08:05:00   
1   @sopansopian5951  2025-10-19 15:48:34   
2          @Di_Wuruk  2025-10-18 11:01:39   
3  @apinlee88channel  2025-10-18 10:59:03   
4      @jarotization  2025-10-18 08:03:25   

                                       teks_komentar  jumlah_likes  \
0  Kl beli LM indent krn stok di toko emas (terke...             0   
1  Ini hanya sifat harga sementara\nEmas bakal na...             1   
2  Aku malah pengen tau perkembangan RENCANA DMO ...             1   
3  Ingat Emas itu Investasi Jangka Panjang ,J

## Melakukan training 300 data dengan SVM

In [4]:
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder

# 1. Membaca data
df = pd.read_csv('hasil_labelling_emas.csv')

# 2. Preprocessing sederhana
def clean_text(text):
    text = str(text).lower()            # Mengubah ke huruf kecil
    text = re.sub(r'http\S+', '', text) # Menghapus URL
    text = re.sub(r'[^\w\s]', '', text) # Menghapus tanda baca & emoji
    text = re.sub(r'\d+', '', text)     # Menghapus angka
    text = text.strip()                 # Menghapus spasi berlebih
    return text

# Terapkan preprocessing ke kolom 'text'
df['clean_text'] = df['text'].apply(clean_text)

# 3. Ekstraksi Fitur (TF-IDF)
vectorizer = TfidfVectorizer(max_features=1000)
X = vectorizer.fit_transform(df['clean_text'])
y = df['label']

# 4. Encoding Label (Mengubah label 'positif', 'negatif' jadi angka)
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# 5. Split Data (80% untuk Training, 20% untuk Testing)
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# 6. Training Model SVM
svm_model = SVC(kernel='linear')
svm_model.fit(X_train, y_train)

# 7. Evaluasi Model
y_pred = svm_model.predict(X_test)

print("Akurasi Model:", accuracy_score(y_test, y_pred))
print("\nLaporan Klasifikasi:\n", classification_report(y_test, y_pred, target_names=le.classes_))

Akurasi Model: 0.35

Laporan Klasifikasi:
               precision    recall  f1-score   support

     negatif       0.36      0.43      0.39        21
      netral       0.24      0.20      0.22        20
     positif       0.44      0.42      0.43        19

    accuracy                           0.35        60
   macro avg       0.35      0.35      0.35        60
weighted avg       0.35      0.35      0.35        60



## Melakukan labeling 700 data dengan model SVM

In [5]:
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC

# 1. Membaca Data
df_train = pd.read_csv('hasil_labelling_emas.csv') # 300 data berlabel
df_target = pd.read_csv('scraping_emas.csv')       # Data yang akan dilabeli

# 2. Fungsi Preprocessing
def clean_text(text):
    text = str(text).lower()
    text = re.sub(r'http\S+', '', text) # Hapus URL
    text = re.sub(r'[^\w\s]', '', text) # Hapus simbol
    text = re.sub(r'\d+', '', text)     # Hapus angka
    text = text.strip()
    return text

df_train['clean_text'] = df_train['text'].apply(clean_text)
df_target['clean_text'] = df_target['teks_komentar'].apply(clean_text)

# 3. Vectorization (TF-IDF)
vectorizer = TfidfVectorizer(max_features=1000)
X_train = vectorizer.fit_transform(df_train['clean_text'])
X_target = vectorizer.transform(df_target['clean_text'])

y_train = df_train['label']

# 4. Training Model SVM
svm_model = SVC(kernel='linear')
svm_model.fit(X_train, y_train)

# 5. Prediksi pada data target
predictions = svm_model.predict(X_target)

# 6. Menyimpan Hasil
df_target['prediksi_label'] = predictions

# Simpan ke CSV baru
df_target.to_csv('hasil_prediksi_scraping.csv', index=False)

print("Proses Selesai. Hasil prediksi disimpan di 'hasil_prediksi_scraping.csv'")
print("\nContoh Hasil:")
print(df_target[['teks_komentar', 'prediksi_label']].head())

Proses Selesai. Hasil prediksi disimpan di 'hasil_prediksi_scraping.csv'

Contoh Hasil:
                                       teks_komentar prediksi_label
0  Kl beli LM indent krn stok di toko emas (terke...         netral
1  Ini hanya sifat harga sementara\nEmas bakal na...        negatif
2  Aku malah pengen tau perkembangan RENCANA DMO ...        positif
3  Ingat Emas itu Investasi Jangka Panjang ,Jd ga...        positif
4  Saatnya turun, Hamas vs Israel damai, Putin se...         netral


In [6]:
!pip install transformers torch pandas

## Melakukan labeling 1000 data secara otomatis dengan LLM

In [7]:
import pandas as pd
from transformers import pipeline

# 1. Setup Model Analisis Sentimen (IndoBERT)
print("Sedang memuat model (mungkin butuh waktu download di awal)...")
sentiment_analysis = pipeline("sentiment-analysis", model="w11wo/indonesian-roberta-base-sentiment-classifier")

# 2. Baca Data
df = pd.read_csv('scraping_emas.csv')

# 3. Fungsi Mapping Label
def get_label_bert(text):
    try:
        # Truncate text jika terlalu panjang (max 512 token)
        result = sentiment_analysis(text[:512])[0]
        label_map = {
            'positive': 'positif',
            'negative': 'negatif',
            'neutral': 'netral'
        }
        return label_map.get(result['label'], 'netral')
    except Exception as e:
        return "netral"

# 4. Proses Labelling
print("Memulai labelling...")
df['label_otomatis'] = df['teks_komentar'].apply(get_label_bert)

# 5. Simpan Hasil
df.to_csv('hasil_labelling_bert.csv', index=False)
print("Selesai! File disimpan sebagai 'hasil_labelling_bert.csv'")
print(df[['teks_komentar', 'label_otomatis']].head())

Sedang memuat model (mungkin butuh waktu download di awal)...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/929 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/328 [00:00<?, ?B/s]

...

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Device set to use cpu


Memulai labelling...
Selesai! File disimpan sebagai 'hasil_labelling_bert.csv'
                                       teks_komentar label_otomatis
0  Kl beli LM indent krn stok di toko emas (terke...        negatif
1  Ini hanya sifat harga sementara\nEmas bakal na...         netral
2  Aku malah pengen tau perkembangan RENCANA DMO ...        positif
3  Ingat Emas itu Investasi Jangka Panjang ,Jd ga...        positif
4  Saatnya turun, Hamas vs Israel damai, Putin se...        negatif


In [8]:
import pandas as pd
from transformers import pipeline
from sklearn.metrics import accuracy_score, classification_report

# MENGHITUNG AKURASI
print("\n--- Memulai Pengujian Akurasi ---")
df_test = pd.read_csv('hasil_labelling_emas.csv')

# Lakukan prediksi pada data test
print("Sedang memprediksi 300 data labelling...")
df_test['prediksi_bert'] = df_test['text'].apply(get_label_bert)

# Bandingkan label asli vs prediksi model
akurasi = accuracy_score(df_test['label'], df_test['prediksi_bert'])
laporan = classification_report(df_test['label'], df_test['prediksi_bert'])

print(f"\n✅ Akurasi Model IndoBERT: {akurasi * 100:.2f}%")
print("\nDetail Laporan Klasifikasi:")
print(laporan)


--- Memulai Pengujian Akurasi ---
Sedang memprediksi 300 data labelling...

✅ Akurasi Model IndoBERT: 41.67%

Detail Laporan Klasifikasi:
              precision    recall  f1-score   support

     negatif       0.46      0.62      0.53       100
      netral       0.32      0.27      0.29       100
     positif       0.44      0.36      0.40       100

    accuracy                           0.42       300
   macro avg       0.41      0.42      0.41       300
weighted avg       0.41      0.42      0.41       300



## Perbandingan model SVM dan LLM

In [9]:
import pandas as pd
from sklearn.metrics import accuracy_score, confusion_matrix

# 1. Membaca Data Hasil Prediksi
try:
    df_llm = pd.read_csv('hasil_labelling_bert.csv')      # Hasil dari kode LLM/IndoBERT
    df_svm = pd.read_csv('hasil_prediksi_scraping.csv')   # Hasil dari kode SVM sebelumnya

    # Pastikan jumlah baris sama
    if len(df_llm) != len(df_svm):
        print("Peringatan: Jumlah data kedua file berbeda!")

    # 2. Membuat DataFrame Perbandingan
    df_compare = pd.DataFrame()
    df_compare['teks'] = df_llm['teks_komentar']
    df_compare['label_llm'] = df_llm['label_otomatis'] # Label dari LLM
    df_compare['label_svm'] = df_svm['prediksi_label'] # Label dari SVM

    # 3. Menghitung Tingkat Kesepakatan (Agreement Rate)
    agreement = accuracy_score(df_compare['label_llm'], df_compare['label_svm'])
    print(f"--- Hasil Perbandingan Model ---")
    print(f"Tingkat Kesepakatan (Agreement Rate): {agreement * 100:.2f}%")

    # 4. Melihat Matriks Perbedaan
    print("\nMatriks Perbandingan (Baris=LLM, Kolom=SVM):")
    print(pd.crosstab(df_compare['label_llm'], df_compare['label_svm']))

    # 5. Melihat Contoh Data yang Berbeda Pendapat
    beda = df_compare[df_compare['label_llm'] != df_compare['label_svm']]
    print(f"\nJumlah perbedaan pendapat: {len(beda)} data")
    print("Contoh data yang berbeda:")
    print(beda.head())

    # Simpan untuk analisis manual
    df_compare.to_csv('perbandingan_llm_vs_svm.csv', index=False)
    print("\nFile perbandingan disimpan sebagai 'perbandingan_llm_vs_svm.csv'")

except FileNotFoundError:
    print("Error: File hasil prediksi tidak ditemukan. Pastikan Anda sudah menjalankan kode labelling LLM dan SVM sebelumnya.")

--- Hasil Perbandingan Model ---
Tingkat Kesepakatan (Agreement Rate): 39.70%

Matriks Perbandingan (Baris=LLM, Kolom=SVM):
label_svm  negatif  netral  positif
label_llm                          
negatif        198     136       93
netral          76     105       93
positif        127      78       94

Jumlah perbedaan pendapat: 603 data
Contoh data yang berbeda:
                                                teks label_llm label_svm
0  Kl beli LM indent krn stok di toko emas (terke...   negatif    netral
1  Ini hanya sifat harga sementara\nEmas bakal na...    netral   negatif
4  Saatnya turun, Hamas vs Israel damai, Putin se...   negatif    netral
6  Inflasi itu nyata... Kelen liat aja harga moto...   negatif   positif
9  Beli emas perhiasan 23 karat tahun 2000an harg...   negatif    netral

File perbandingan disimpan sebagai 'perbandingan_llm_vs_svm.csv'


## Mendownload hasil model SVM dan LLM

In [10]:
import pandas as pd

# 1. Membaca kedua file hasil prediksi
try:
    df_svm = pd.read_csv('hasil_prediksi_scraping.csv')
    df_llm = pd.read_csv('hasil_labelling_bert.csv')

    print("File berhasil dibaca.")
    print(f"Jumlah data SVM: {len(df_svm)}")
    print(f"Jumlah data LLM: {len(df_llm)}")

    # 2. Pengecekan Konsistensi Data
    if len(df_svm) != len(df_llm):
        print("PERINGATAN: Jumlah baris data tidak sama. Penggabungan mungkin tidak akurat.")

    # 3. Membuat DataFrame Gabungan (Base dari SVM)
    df_gabungan = df_svm.copy()

    if 'prediksi_label' in df_gabungan.columns:
        df_gabungan.rename(columns={'prediksi_label': 'label_svm'}, inplace=True)

    # 4. Mengambil label dari file LLM
    if 'label_otomatis' in df_llm.columns:
        df_gabungan['label_llm'] = df_llm['label_otomatis']

    # 5. Memilih HANYA kolom yang diinginkan
    kolom_yang_dipilih = ['teks_komentar', 'label_svm', 'label_llm']

    # Pastikan kolom-kolom tersebut ada sebelum di-filter
    try:
        df_gabungan = df_gabungan[kolom_yang_dipilih]
    except KeyError as e:
        print(f"Error: Kolom {e} tidak ditemukan dalam data.")
        raise # Hentikan proses jika kolom utama hilang

    # 6. Menyimpan File Gabungan
    output_filename = 'data_self_training.csv'
    df_gabungan.to_csv(output_filename, index=False)

    print(f"\nSukses! File gabungan disimpan sebagai: {output_filename}")

    # 7. Menampilkan Preview
    print("\nPreview 5 data pertama:")
    print(df_gabungan.head())

    # 8. Statistik Singkat
    total_sepakat = (df_gabungan['label_svm'] == df_gabungan['label_llm']).sum()
    persen_sepakat = (total_sepakat / len(df_gabungan)) * 100

    print(f"\nStatistik Tambahan:")
    print(f"Kedua model sepakat pada {total_sepakat} dari {len(df_gabungan)} data ({persen_sepakat:.2f}%)")

except FileNotFoundError:
    print("Error: Salah satu file tidak ditemukan. Pastikan 'hasil_prediksi_scraping.csv' dan 'hasil_labelling_bert.csv' ada di folder yang sama.")
except Exception as e:
    print(f"Terjadi kesalahan: {e}")

File berhasil dibaca.
Jumlah data SVM: 1000
Jumlah data LLM: 1000

Sukses! File gabungan disimpan sebagai: data_self_training.csv

Preview 5 data pertama:
                                       teks_komentar label_svm label_llm
0  Kl beli LM indent krn stok di toko emas (terke...    netral   negatif
1  Ini hanya sifat harga sementara\nEmas bakal na...   negatif    netral
2  Aku malah pengen tau perkembangan RENCANA DMO ...   positif   positif
3  Ingat Emas itu Investasi Jangka Panjang ,Jd ga...   positif   positif
4  Saatnya turun, Hamas vs Israel damai, Putin se...    netral   negatif

Statistik Tambahan:
Kedua model sepakat pada 397 dari 1000 data (39.70%)
